In [1]:
import sys
import numpy as np
from matplotlib import pyplot as plt
import h5py
from collections import OrderedDict

%matplotlib notebook

In [2]:
import qcodes as qc

# Setup notes

For this example to run, you need to have set up a labpython config file. There is a template in labpythonQDelft/templates.

In [9]:
from importlib import reload
from pystationq.qctools import instruments as instools
reload(instools)

import labpythonconfig as cfg
reload(cfg)

from pystationq.data import data_storage
reload(data_storage)
from pystationq.data.data_storage import Data, GridData

from pystationq.experiment import measurement
reload(measurement)
from pystationq.experiment.measurement import Parameter, BaseMeasurement

In [22]:
from pysweep.data_storage import base_storage
reload(base_storage)
from pysweep.data_storage.base_storage import BaseStorage, DataSet

from pysweep import sweep_object
reload(sweep_object)
from pysweep.sweep_object import BaseSweepObject, sweep, ChainSweep

In [11]:
station = qc.Station()

In [12]:
class NameSpace:
    pass

namespace = NameSpace()

In [13]:
class DummyMeasurement(BaseMeasurement):
    def __init__(self, station, namespace):
        super().__init__(station, namespace)
        
        self.add_parameter('start', Parameter)
        self.add_parameter('stop', Parameter)
        self.add_parameter('npts', Parameter)
    
    def measure(self):        
        # set up loop
        xpoints = np.linspace(self.start(), self.stop(), self.npts())
        ypoints = np.linspace(self.start(), self.stop(), self.npts())
                
        # now populate in a primitive loop
        for x in xpoints:
            for y in ypoints:
                
                # acquire some data
                z = np.cos(x) * np.cos(y) * np.linspace(0,1,3)

                # add to dataset
                self.data.add(OrderedDict(
                        {'idx' : {'value' : np.arange(z.size).reshape(z.shape), 'independent_parameter' : True, },
                         'y' : {'value' : y, 'independent_parameter' : True, 'unit' : 'V'},
                         'x' : {'value' : x, 'independent_parameter' : True, 'unit' : 'V'},
                         'z' : {'value' : z },
                        }
                ))
                    

m = DummyMeasurement(station, namespace)
m.start(0)
m.stop(10)
m.npts(3)
m.run()

In [14]:
m.data['z'].dtype

dtype([('idx', '<i4', (3,)), ('y [V]', '<f8', (1,)), ('x [V]', '<f8', (1,)), ('z', '<f8', (3,))])

In [29]:
voltage1 = 0
voltage2 = 0

def getv1():
    global voltage1
    return voltage1

def setv1(v):
    global voltage1
    voltage1 = v

def getv2():
    global voltage2
    return voltage2

def setv2(v):
    global voltage2
    voltage2 = v

v1 = qc.Parameter('voltage1', get_cmd=getv1, set_cmd=setv1, unit='V')
v2 = qc.Parameter('voltage2', get_cmd=getv2, set_cmd=setv2)

station = qc.Station(v1, v2)

In [30]:
BaseSweepObject.parameter_log_format(v1, 0)

OrderedDict([('voltage1',
              {'independent_parameter': True, 'unit': 'V', 'value': 0})])

In [28]:
class PysweepGrid(BaseMeasurement):
    
    data_cls = GridData
    
    @staticmethod
    def _get_name(param):
        pass
    
    def measure(self):
        swp = []
        coords = []
        for p, vals in self.sweep:
            swp.append(sweep(p, vals))
            
            
        print(coords)
        swp.append(self.measure_datapoint)
        
        i = 0
        for rec in ChainSweep([tuple(swp)]):
            if not i:
                for n, v in rec.items():
                    pass
                
            self.data.add(rec)
            i += 1
    

class SimpleGrid(PysweepGrid):
    
    def measure_datapoint(self, station, namespace):
        scale = np.linspace(0,1,3)
        return OrderedDict({
            'scale' : {'value' : scale, 'independent_parameter' : True, },
            'data' : {'value' : (self.station.voltage1() + self.station.voltage2()**2) * scale},
        })
    
    def sweep(self):
        return
    
    
func = lambda station, namespace, x : {}
    
m = SimpleGrid(station, namespace)
m.sweep = [
#     (m.station.voltage1, np.linspace(0,1,3)), 
    (func, np.linspace(0,1,3)),
    (m.station.voltage2, np.linspace(0,2,5)),
]
m.run()

[]


TypeError: 'NoneType' object is not iterable

In [18]:
fn = m.datafilepath
print(fn)

c:\data\2017-12\2017-12-28\2017-12-28_SimpleGrid-#0000.hdf5


In [24]:
m.data._pages['data'].dtype

dtype([('scale', '<f8', (3,)), ('voltage2', '<f8', (1,)), ('voltage1', '<f8', (1,)), ('data', '<f8', (3,))])